# Decision Tree

## Imports

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from collections import Counter
from collections import deque
from sklearn import datasets
from sklearn import tree
import numpy as np
import pydotplus
import math
import os

### Utility functions

In [2]:
def get_most_occurring_feature(Y):
    counter = Counter(Y)
    k, v = counter.most_common(1)[0]
    return k
    
    
""" functions to calculate impurity and gain """

# i) method - information gain
def entropy(Y):
        """ Y :  List(data) representing classes.
            entropy = -∑pi*log(pi) 
        """
        frequency = Counter(Y)
        entropy = 0
        total = len(Y)
        for i in frequency:
            p = frequency[i]/total
            entropy -= (p)*math.log2(p)
        return entropy
    
    
def gain_ratio(previous_classes, current_classes):
        """ previous_classes (list(int)): represents classes before split.
            current_classes (list(list(int): A list of lists of classes after split according to current feature.
        """
        original_impurity = entropy(previous_classes) #-----> original_impurity represents entropy before splitting
        current_impurity = 0  #-----------------------------> entropy after splitting upon the selected feature
        split_info = 0  #-----------------------------------> penalty
        previous_size = len(previous_classes)
        for class_i in current_classes:
            current_size = len(class_i)
            current_impurity += (current_size/previous_size)*entropy(class_i)
            split_info += (-current_size/previous_size)*math.log2(current_size/previous_size)

        # edge case to handle 0 division error
        if split_info == 0 :
            return math.inf

        info_gain = original_impurity - current_impurity
        return (info_gain / split_info) #-------------------> gain ratio
 


    
# ii) method - gini index

def gini_index(Y):
    prob_sum = 0 #------------------------------------------> variable to store ∑pi^2 value
    freq_map = Counter(Y)
    n = len(Y) #--------------------------------------------> count of all classes
    for class_i in freq_map:
        prob_sum += (freq_map[class_i]/n)**2 #--------------> pi^2 => (count of ith class/count of all classes)^2

    return 1 - prob_sum # ----------------------------------> Gini_index = 1 − ∑pi^2
    
    
def gini_gain(previous_classes, current_classes):
    """ previous_classes (list(data)): Vector of classes.
        current_classes (list(list(data): A list of lists of classes divided by the current feature.
    """
    original_impurity = gini_index(previous_classes)
    current_impurity = 0 #---------------------------------> stores weighted sum of gini impurities of every split sections
    previous_len = len(previous_classes)
    
    # edge case - if one of the splitted class is empty
    if len(current_classes[0]) == 0 or len(current_classes[1]) == 0:
        return 0

    for current_class in current_classes:
        weightage = len(current_class)/previous_len
        current_impurity += weightage * gini_index(current_class)

    return original_impurity - current_impurity #----------> total gain



""" split related funtions"""

# function to split the dataset in 2 parts according to feature values 
def split_by_feature(X,Y, feature, split_val):
    x_slices = [[],[]] # ----------------------------------> stores left split and right split of X
    y_slices = [[],[]] # ----------------------------------> stores left split and right split of Y
    
    for row in range(len(X)):
        if X[row, feature]<= split_val:
            col = 0 #--------------------------------------> left split index
        else:
            col = 1 #--------------------------------------> right split index
        x_slices[col].append(X[row,:])
        y_slices[col].append(Y[row])

    return x_slices, y_slices


# function to search for best split value using split_by_feature function
def best_split(X, Y, feature, gini):
    """ feature : column index of X which represents the splitting feature
        gini : boolean value to decide the gain method (true:gini_index, false:entropy)
    """
    values = list(set(X[:, feature])) # -------------------> set of unique feature values
    max_gain = -1
    best_x_split = None # ---------------------------------> stores the best split of X depending on max_gain
    best_y_split = None # ---------------------------------> stores the best split of Y depending on max_gain
    best_split_val = -1 #----------------------------------> stores the best feature value which yields best gain
    for i in range(1,len(values)):
        val = (values[i-1]+values[i])/2 # --------------------> split value
        x_split, y_split = split_by_feature(X, Y, feature, val)
        if gini:
            gain = gini_gain(Y, y_split)
        else:
            gain = gain_ratio(Y, y_split)
        if gain>max_gain:
            max_gain = gain
            best_x_split = x_split
            best_y_split = y_split
            best_split_val = val
    return best_x_split, best_y_split, max_gain, best_split_val

### Decision Tree Node:

In [3]:
class Node:
    def __init__(self, deciding_feature=None, split_val=None, class_mark=None, entropy=0):
        self.left = None #---------------------------------> left child initialized as None
        self.right = None #--------------------------------> right child initialized as None
        self.deciding_feature = deciding_feature  #--------> feature with max gain
        self.split_val = split_val
        self.deciding_func = lambda x: (x[deciding_feature]<=split_val)
        self.class_mark = class_mark  #--------------------> holds output class value for leaf node otherwise None
        self.entropy = entropy #---------------------------> impurity amongst output
    
    def decide(self, x):
        if self.class_mark is not None: #------------------> only leaf contains a class mark
            return self.class_mark 
        if self.deciding_func(x):
            return self.left.decide(x)
        return self.right.decide(x)

### Creating the class for decision tree

In [4]:
class DecisionTree:
    
    def __init__(self,depth_limit = math.inf):
        self.root = None
        self.depth_limit = depth_limit
        
    
    def fit(self, X,Y, feature_names= None, metric="gain_ratio"):
        """ function to build decision tree according to given training data
            and store the root Node
        """
        # creating feature names if not passed
        if feature_names is None:
            feature_names = ["X["+str(i)+"]" for i in range(len(X[0]))] #-----> storing feature name as X[i]
        self.feature_names = feature_names
        
        feature_indexes = [i for i in range(len(X[0]))] #---------------------> using numbers in actual functions is easier
        
        self.classes = list(set(Y))
        
        self.gini = False #---------------------------------------------------> represents gain calculation method (True: gini_index, False: info gain)
        if (metric.lower()=="gini_index"):
            self.gini = True
            
        self.root = self.generate(X,Y, feature_indexes)
        
        
            
            
    def generate(self, X,Y,features, level = 0):
        print("Level",level)
        
        # Edge Case 1 - pure output set
        if len(set(Y)) == 1:
            output = Y[0]
            print("Count of",Y[0],"=",len(Y))
            if self.gini:
                print("Current gini impurity is =  0.0")
            else:
                print("Current Entropy is =  0.0")
            print("Reached leaf Node\n")
            return Node(class_mark=output)
        
        # common part
        # print counts of classes
        freq_map = Counter(Y)
        for i in freq_map:
            print("Count of",i,"=",freq_map[i])
        # print current impurity
        impurity = 0
        if self.gini:
            impurity = gini_index(Y)
            print("Current gini impurity is =", impurity)
        else:
            impurity = entropy(Y)
            print("Current Entropy is =", impurity) 
        
        
        # edge case 2 and 3 - (If we have run out of features to split upon) or (max depth limit reached)
        if len(features) == 0 or level >= self.depth_limit:        
            print("Reached leaf Node\n")
            return Node(class_mark=get_most_occurring_feature(Y), entropy=impurity)
    
        # intermediate node
        max_gain = -1
        best_feature = -1 
        best_x_plit = None
        best_y_split = None
        best_split_val = -1
        X = np.array(X) # for ease of subscripting
        # brute force search for best splitting feature
        for feature in features:
            x_split, y_split, gain, split_val = best_split(X, Y, feature, self.gini)              
            if gain>max_gain:
                max_gain = gain
                best_feature = feature
                best_y_split = y_split
                best_x_split = x_split
                best_split_val = split_val
                
        # creating the node with best feature and corresponding split value
        node = Node(best_feature, best_split_val, entropy=impurity)
        
        # last line to print for an intermediate node
        if self.gini:
            print("Splitting on feature",self.feature_names[best_feature] ,"with gini_gain", max_gain,"\n")
        else:
            print("Splitting on feature",self.feature_names[best_feature] ,"with gain ratio", max_gain,"\n")
        
        """ features.remove(best_feature) """ 
        # commented out the code for removing feature for better accuracy
        
        # adding left and right child and creating the tree recursively
        node.left = self.generate(best_x_split[0], best_y_split[0], features, level+1)
        node.right = self.generate(best_x_split[1], best_y_split[1], features, level+1)
        
        return node
            
    def predict(self, X_test):
        """ X_test : list(list(int)) 
            y_pred : prediction based on row-wise X_test data
        """
        y_pred = []
        for X_i in X_test:
            y_pred.append(self.root.decide(X_i))
            
        return np.array(y_pred)
    
    
    def confusion_matrix(self, y_pred, y_true):
        """ y_pred : list(int) representing prediction classes
            y_true : list(int) representing true classes
            both contains values from 0 to n-1
        """
        n = len(self.classes)
        j = 0
        mat = np.array([0]*(n*n)).reshape(n,n) # -------------------------------> confusion matrix 
        
        for k in range(len(y_true)):
            i = self.classes.index(y_true[k]) #---------------------------------> row represents true class
            j = y_pred[k] #-----------------------------------------------------> col represents prediction class
            mat[i][j] += 1

        return np.array(mat)
    
    
    def node_string(self, node):
        """ returns a string which represents the node in a dot graphviz format
        """
        split_feature = None
        if node.deciding_feature is not None:
            split_feature = self.feature_names[node.deciding_feature]
        class_name = "no output"
        impurity = round(node.entropy,6)
        if node.class_mark is not None:
            class_name = self.classes[node.class_mark]
        return "\n{} [label=\"Split feature : {}\\nNode impurity : {}\\nNode output : {}\" ];".format(id(node),split_feature,impurity,class_name)
                
            
    def to_pdf(self,filename=None):
        """ returns the tree as dot data after saving it as a pdf
        """
        dot_data = '''digraph Tree {
        node [shape=box] ;'''
        
        queue = deque() #--------------------------------> to perform level order traversal
        
        # adding the root
        r = self.root
        queue.append(r)
        dot_data = dot_data + self.node_string(r)
        
        # Doing LEVEL ORDER traversal in the tree (using a queue)
        while len(queue) != 0 :
            node = queue.popleft()
            parent_split_feature = None
            if node.deciding_feature is not None:
                parent_split_feature = self.feature_names[node.deciding_feature]
            if node.left is not None:                
                # Creating left child
                dot_data = dot_data + self.node_string(node.left) 
                # Connecting parent node with left child
                dot_data = dot_data + "\n{} -> {} [ label=\"{}<={}\"]; ".format(id(node),id(node.left), parent_split_feature, node.split_val)
                # Adding left child node to queue
                queue.append(node.left)
                
            if node.right is not None:                
                # Creating right child 
                dot_data = dot_data + self.node_string(node.right) 
                # Connecting parent node with right child
                dot_data = dot_data + "\n{} -> {} [ label=\"{}>{}\"]; ".format(id(node),id(node.right), parent_split_feature, node.split_val)
                # Adding right child node to queue
                queue.append(node.right)
        
        dot_data = dot_data + "\n}"

        if filename != None:    
            graph = pydotplus.graph_from_dot_data(dot_data)
            print(graph)
            graph.write_pdf(filename)    
        
        return dot_data

In [5]:
# example tree
clf = DecisionTree()
clf.fit([[0,0],[0,1],[1,0],[1,1]],[0,1,1,1]) # training data for OR gate

Level 0
Count of 0 = 1
Count of 1 = 3
Current Entropy is = 0.8112781244591328
Splitting on feature X[0] with gain ratio 0.31127812445913283 

Level 1
Count of 0 = 1
Count of 1 = 1
Current Entropy is = 1.0
Splitting on feature X[1] with gain ratio 1.0 

Level 2
Count of 0 = 1
Current Entropy is =  0.0
Reached leaf Node

Level 2
Count of 1 = 1
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 1 = 2
Current Entropy is =  0.0
Reached leaf Node



In [6]:
# testing on the training data for OR gate
clf.confusion_matrix(clf.predict([[0,0],[0,1],[1,0],[1,1]]),[0,1,1,1])

array([[1, 0],
       [0, 3]])

In [7]:
iris_data = datasets.load_iris()
X = iris_data.data
Y = iris_data.target
x_train, x_test, y_train, y_test = train_test_split(X, Y)
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [8]:
clf1 = DecisionTree() # clf1 : my model using info gain
clf1.fit(x_train, y_train,feature_names=iris_data.feature_names)

Level 0
Count of 2 = 38
Count of 1 = 36
Count of 0 = 38
Current Entropy is = 1.5844996446144277
Splitting on feature petal length (cm) with gain ratio 1.0 

Level 1
Count of 0 = 38
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 2 = 38
Count of 1 = 36
Current Entropy is = 0.9994730201859836
Splitting on feature petal length (cm) with gain ratio 0.7949533984153239 

Level 2
Count of 1 = 36
Count of 2 = 3
Current Entropy is = 0.39124356362925566
Splitting on feature petal width (cm) with gain ratio 0.38328097226983665 

Level 3
Count of 1 = 33
Current Entropy is =  0.0
Reached leaf Node

Level 3
Count of 1 = 3
Count of 2 = 3
Current Entropy is = 1.0
Splitting on feature sepal width (cm) with gain ratio 1.0 

Level 4
Count of 2 = 3
Current Entropy is =  0.0
Reached leaf Node

Level 4
Count of 1 = 3
Current Entropy is =  0.0
Reached leaf Node

Level 2
Count of 2 = 35
Current Entropy is =  0.0
Reached leaf Node



In [9]:
clf2 = DecisionTreeClassifier() # standard model of sklearn
clf2.fit(x_train, y_train)

DecisionTreeClassifier()

In [10]:
print("Testing information gain model against standard model:\n")
print("difference:\n", clf1.predict(x_test)-clf2.predict(x_test),"\n")
print("my model:\n", clf1.confusion_matrix(clf1.predict(x_test),y_test),"\n")
print("standard model:\n", confusion_matrix(y_test, clf2.predict(x_test)))

Testing information gain model against standard model:

difference:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0] 

my model:
 [[12  0  0]
 [ 0 10  4]
 [ 0  0 12]] 

standard model:
 [[12  0  0]
 [ 0 10  4]
 [ 0  0 12]]


In [11]:
# using gini index as metric 
clf3 = DecisionTree()
clf3.fit(x_train, y_train, feature_names=iris_data.feature_names, metric="gini_index")

Level 0
Count of 2 = 38
Count of 1 = 36
Count of 0 = 38
Current gini impurity is = 0.666454081632653
Splitting on feature petal length (cm) with gini_gain 0.33633825151682284 

Level 1
Count of 0 = 38
Current gini impurity is =  0.0
Reached leaf Node

Level 1
Count of 2 = 38
Count of 1 = 36
Current gini impurity is = 0.4996347699050402
Splitting on feature petal length (cm) with gini_gain 0.42479069506096545 

Level 2
Count of 1 = 36
Count of 2 = 3
Current gini impurity is = 0.14201183431952646
Splitting on feature petal width (cm) with gini_gain 0.06508875739644954 

Level 3
Count of 1 = 33
Current gini impurity is =  0.0
Reached leaf Node

Level 3
Count of 1 = 3
Count of 2 = 3
Current gini impurity is = 0.5
Splitting on feature sepal width (cm) with gini_gain 0.5 

Level 4
Count of 2 = 3
Current gini impurity is =  0.0
Reached leaf Node

Level 4
Count of 1 = 3
Current gini impurity is =  0.0
Reached leaf Node

Level 2
Count of 2 = 35
Current gini impurity is =  0.0
Reached leaf Node


In [12]:
print("Testing gini index model against standard model:\n")
print("difference:\n", clf3.predict(x_test)-clf2.predict(x_test),"\n")
print("my model:\n", clf3.confusion_matrix(clf3.predict(x_test),y_test),"\n")
print("standard model:\n", confusion_matrix(y_test, clf2.predict(x_test)))

Testing gini index model against standard model:

difference:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0] 

my model:
 [[12  0  0]
 [ 0 10  4]
 [ 0  0 12]] 

standard model:
 [[12  0  0]
 [ 0 10  4]
 [ 0  0 12]]


In [13]:
os.environ['PATH'] = os.environ['PATH']+';'+os.environ['CONDA_PREFIX']+r"\Library\bin\graphviz"
clf.to_pdf("OR.pdf") # --------------> metric - information gain
clf1.to_pdf("Iris_info_gain.pdf") # -> metric - information gain
clf3.to_pdf("Iris_gini_indx.pdf") # -> metric - gini index

'digraph Tree {\n        node [shape=box] ;\n2334821344352 [label="Split feature : petal length (cm)\\nNode impurity : 0.666454\\nNode output : no output" ];\n2334821385792 [label="Split feature : None\\nNode impurity : 0\\nNode output : 0" ];\n2334821344352 -> 2334821385792 [ label="petal length (cm)<=2.95"]; \n2334821387760 [label="Split feature : petal length (cm)\\nNode impurity : 0.499635\\nNode output : no output" ];\n2334821344352 -> 2334821387760 [ label="petal length (cm)>2.95"]; \n2334767985712 [label="Split feature : petal width (cm)\\nNode impurity : 0.142012\\nNode output : no output" ];\n2334821387760 -> 2334767985712 [ label="petal length (cm)<=4.800000000000001"]; \n2334821742096 [label="Split feature : None\\nNode impurity : 0\\nNode output : 2" ];\n2334821387760 -> 2334821742096 [ label="petal length (cm)>4.800000000000001"]; \n2334821737616 [label="Split feature : None\\nNode impurity : 0\\nNode output : 1" ];\n2334767985712 -> 2334821737616 [ label="petal width (cm)